In [1]:
import pandas as pd
import numpy as np 
npNaN = np.nan
import pandas_ta as ta
import matplotlib.pyplot as plt
import config
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders
from datetime import datetime, timezone
import time 

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pandas_ta/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [ ]:
#setting up API connection 
client = API(access_token=config.OANDA_API_KEY, environment=config.ENVIRONMENT)


In [ ]:
#defining timeframe and instrument 
timeframe = "M1"
instrument = "GBP_JPY"

<module 'time' (built-in)>


In [4]:
#defining the parameters for the request/instrument 
#Trading bot will regularly request this data so we need to wrap it in a functiuon 
def get_candles(tf):
    params = {
    "granularity": tf,
    "price": "A" # A for asking price
}
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
#Because this is a dictionary, we can access the candles directly
    candles = client.request(r)["candles"]
    candles

#So now we're going to put it into a dataframe
#Convert to pandas dataframe

    data = []
    for c in candles:
        if c["complete"]:
            data.append({
                "time": c["time"],
                "open": float(c["ask"]["o"]),
                "high": float(c["ask"]["h"]),
                "low": float(c["ask"]["l"]),
                "close": float(c["ask"]["c"])
         })
    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"])
    return df
print(get_candles(timeframe).head())




                       time     open     high      low    close
0 2025-08-21 00:32:00+00:00  198.385  198.392  198.371  198.386
1 2025-08-21 00:33:00+00:00  198.387  198.394  198.374  198.383
2 2025-08-21 00:34:00+00:00  198.385  198.404  198.384  198.399
3 2025-08-21 00:35:00+00:00  198.397  198.405  198.372  198.378
4 2025-08-21 00:36:00+00:00  198.378  198.391  198.361  198.370


In [5]:
#Function to calculate indicators
def calculate_indicators(df):
    #EMA
    df["EMA_7"] = ta.ema(df["close"], length=7)
    df["EMA_14"] = ta.ema(df["close"], length=14)

    #ATR
    df["ATR_14"] = ta.atr(df["high"], df["low"], df["close"], length = 14)

    return df

In [6]:
#Function to place order 
def place_sell_order(stop_loss, take_profit):
    data = {
           "order":{ 
            "instrument": instrument, 
            "units": -5000,  # Number of units to trade
            "type": "MARKET",
            "stopLossOnFill": {"price": f"{stop_loss:.3f}"}, #.3f formats the float to 3 decimal places which is how many OANDA uses for GBP/JPY
            "takeProfitOnFill": {"price": f"{take_profit:.3f}"}}
           }
    r = orders.OrderCreate(config.OANDA_ACCOUNT_ID, data=data)
    client.request(r)
    print(f"Placed  sell order for {instrument} with stop loss at {round(stop_loss, 3)} and take profit at {round(take_profit, 3)}")

def place_buy_order(stop_loss, take_profit):
    data = {
           "order":{ 
            "instrument": instrument, 
            "units": 5000,  # Number of units to trade
            "type": "MARKET",
            "stopLossOnFill": {"price": f"{stop_loss:.3f}"}, #.3f formats the float to 3 decimal places which is how many OANDA uses for GBP/JPY
            "takeProfitOnFill": {"price": f"{take_profit:.3f}"}}
           }
    r = orders.OrderCreate(config.OANDA_ACCOUNT_ID, data=data)
    client.request(r)
    print(f"Placed buy order for {instrument} with stop loss at {round(stop_loss, 3)} and take profit at {round(take_profit, 3)}")
    #Use the as a visual confirmation that the order was placed, can check with calling the function: "Place_order(190, 200)"
 


In [7]:
#Function to check for EMA crossovers
#we need to check if the 8 EMA if above or below the previous 5 EMA to dectect a crossover
#So we need to fetch both of these candles from the dateframe
def ema_crossover(df):
    #whatever my stop loss is, my tp will be 1.5 times that
    tp_ratio = 1.5
    atr_multiplier = 1.5  # ATR multiplier for stop loss calculation
    #check if the 5EMA crosses above the 8EMA 
    last_candle = df.iloc[-1]
    previous_candle = df.iloc[-2]
    #Crossover buy signal (EMA 5 crosses above EMA 8)
    if last_candle['EMA_7'] > last_candle['EMA_14'] and previous_candle['EMA_7'] < previous_candle['EMA_14']:
        print("Buy signal: EMA 7 crossed above EMA 14")
        #where will SL and Tp be? well since its bassed on the last candle we can use that 
        entry_price = last_candle['close']
        stop_loss = ((entry_price + last_candle['ATR_14']) * atr_multiplier) 
        stop_loss_distance = entry_price - stop_loss
        take_profit = entry_price + (stop_loss_distance * tp_ratio)
        place_buy_order(stop_loss, take_profit)
    elif last_candle['EMA_7'] < last_candle['EMA_14'] and previous_candle['EMA_7'] > previous_candle['EMA_14']:
        print("Sell signal: EMA 7 crossed below EMA 14")
        entry_price = last_candle['close']
        stop_loss = ((entry_price + last_candle['ATR_14']) * atr_multiplier)
        stop_loss_distance = stop_loss - entry_price
        take_profit = entry_price - (stop_loss_distance * tp_ratio)
        place_sell_order(stop_loss, take_profit)
    else:
        print("Stratergy conditions not met")


In [8]:


#So we have a trading bot that is connected to the OANDA API and can get the price of GBP/JPY in 15 minute intervals

In [ ]:
#Create a place to combine all of this functionality into a central place for the trading bot
def run_bot():
    print("starting trading bot")
    last_checked = None
    #get the same data but its much easier to access it
    while True:
        current_time = datetime.now(timezone.utc)
        #check for a new candle every 15 minutes
        #if current_time.minute % 15 == 0 and current_time.second < 10: 
        if current_time.minute % 1 == 0 and current_time.second < 10:
            #check if the 15-min nterval has changed since the last check
            #make sure we arent checking the same candle multiple times
            if last_checked != current_time.minute:
                print(f"Current time: {current_time}")
                print("checking for new signlas")
                price = get_candles(timeframe)
                price = calculate_indicators(price)
                ema_crossover(price)
                last_checked = current_time.minute #update to prevent re-triggering 
        time.sleep(1)
run_bot()
#check if the order placing works
#place_order(195, 205)

starting trading bot
Current time: 2025-08-21 08:52:00.545508+00:00
checking for new signlas
Stratergy conditions not met


In [ ]:
#Now we need to get a stratergy by setting up some indicators 

In [ ]:
#Plots the graph 
plt.plot(df['close'])
plt.show()